In [1]:
%pip install transformers optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pre-trained GPT-2 model
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the model is on CPU
model.to('cpu')

# Apply dynamic quantization to the model (only quantize the Linear layers)
quantized_model = torch.quantization.quantize_dynamic(
    model,  # The model to be quantized
    {torch.nn.Linear},  # Quantize only the Linear layers
    dtype=torch.qint8  # Use INT8 for quantization
)

# Iterate through all modules and submodules of the quantized model
for module in quantized_model.modules():
    # Check if the module has parameters
    if hasattr(module, 'weight') and module.weight is not None:
        # Check if the weight tensor is on the 'meta' device
        if module.weight.device.type == 'meta':
            # If on 'meta' device, move it to CPU and replace the module's weight
            module.weight = torch.nn.Parameter(module.weight.data.to('cpu'))

    if hasattr(module, 'bias') and module.bias is not None:
        if module.bias.device.type == 'meta':
            module.bias = torch.nn.Parameter(module.bias.data.to('cpu'))

# After moving parameters, save the quantized model to disk
quantized_model.save_pretrained('./quantized_gpt2')

# Now, test by loading the model again
quantized_model = AutoModelForCausalLM.from_pretrained('./quantized_gpt2')

# Generate some text using the quantized model
input_ids = tokenizer.encode("Hello, how are you?", return_tensors='pt')
output = quantized_model.generate(input_ids, max_length=50)

# Decode and print the output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

AttributeError: 'function' object has no attribute 'device'

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pre-trained GPT-2 model
model_name = 'gpt2-xl'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the model is on CPU
model.to('cpu')

# Check if the model has any parameters on a non-CPU device
for param in model.parameters():
    if param.device.type != 'cpu':
        print(f"Found parameter on {param.device.type} device!")
    else:
        print("All parameters are on the CPU!")

# Apply dynamic quantization to the model (only quantize the Linear layers)
quantized_model = torch.quantization.quantize_dynamic(
    model,  # The model to be quantized
    {torch.nn.Linear},  # Quantize only the Linear layers
    dtype=torch.qint8  # Use INT8 for quantization
)

# Verify that the quantized model does not have any "meta" device issues
for param in quantized_model.parameters():
    if param.device.type == "meta":
        print(f"Parameter on meta device: {param}")
        param.data = param.data.to('cpu')  # Move it to CPU

# After quantization, check that all parameters are on CPU
for param in quantized_model.parameters():
    if param.device.type != 'cpu':
        print(f"Found parameter on {param.device.type} device after quantization!")
    else:
        print("All parameters are on CPU after quantization!")

# Save the quantized model to disk
quantized_model.save_pretrained('./quantized_gpt2')

# Now, test by loading the model again
quantized_model = AutoModelForCausalLM.from_pretrained('./quantized_gpt2')

# Generate some text using the quantized model
input_ids = tokenizer.encode("Hello, how are you?", return_tensors='pt')
output = quantized_model.generate(input_ids, max_length=50)

# Decode and print the output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All parameters are on the CPU!
All para

In [24]:
import torch

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,  # The model to be quantized
    {torch.nn.Linear},  # Specify layers to quantize, typically Linear layers
    dtype=torch.qint8  # Use INT8 for quantization
)
# device = torch.device("meta" if torch.cuda.is_available() else "cpu")
# print(device)
# torch.cuda.set_device(device)
quantized_model = quantized_model.to('cuda')
# Save the quantized model for later use
quantized_model.save_pretrained('./quantized_gpt2')


AttributeError: 'torch.dtype' object has no attribute 'device'

In [22]:
%pip show torch

Name: torch
Version: 2.5.1+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, optimum, peft, sentence-transformers, timm, torchaudio, torchvision


In [3]:
%pip install optimum

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load GPT-2 model and tokenizer
model_name = 'gpt2-medium'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to('cpu')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [10]:
import torch

# Apply dynamic quantization to the model
quantized_model = torch.quantization.quantize_dynamic(
    model,  # The model to be quantized
    {torch.nn.Linear},  # Quantize only the Linear layers
    dtype=torch.qint8  # Quantize to INT8
)

# Save the quantized model
quantized_model.save_pretrained('./quantized_gpt2')


AttributeError: 'torch.dtype' object has no attribute 'device'